# 批次

目前为止，在模型训练的过程中，我们每次取一条数据进行迭代。事实上，得益于NumPy数组的矢量并行运算能力，我们可以一次把多条数据送入迭代。这种迭代方式称为**批次梯度下降**（Batch Gradient Descent）。相对应地，每次一条数据的迭代方式称为**随机梯度下降**（Stochastic Gradient Descent）。

批次又可以分为：
* **全批次**（Full Batch）：就是一次把所有训练数据全部送入迭代。实践中很少使用全批次进行训练，因为需要大量的内存，运算速度会非常缓慢。
* **小批次**（Mini-batch）：就是每次把数条数据送入迭代。小批次充分地利用了NumPy（或者其他计算库）的矢量并行运算能力，是目前常规的模型训练方式。

全批次每次迭代都可以从全部数据进行学习，因此可以保持正确的收敛方向，所以全批次训练的收敛速度较快。相对应的，随机梯度下降每次迭代只能从一条数据进行学习，因此梯度变化较大，收敛方向左右摇摆，所以随机梯度下降的收敛速度较慢。而小批次处于两者之间比较平衡的位置，这也是小批次训练比较常用的另一个原因。

## 泛化

**泛化**（Generalization）是评价模型训练效果的一个重要指标。即模型可以对新数据进行合理、有效的推理的能力。简单地讲，就是一个在训练数据上表现良好的模型，是否可以在验证数据上同样有好的表现。

通常来讲，泛化能力来自于对多样化的数据进行学习。因此随机梯度下降的训练方式通常可以带来更好的模型泛化能力；小批次训练次之；全批次训练更次之。

泛化能力差一般有两种表现：
* **欠拟合**（Underfitting）：模型在训练数据上表现就很差，在验证数据上表现也很差。通常是因为训练数据不足，或者模型过于简单。
* **过拟合**（Overfitting）：模型在训练数据上接近优异，但在验证数据上上表现糟糕。通常是因为模型过于复杂，从训练数据中学习到了过多的、没有代表性的细节。

In [1]:
import numpy as np

## 数据集

### 训练数据：特征、标签

In [2]:
train_features = np.array([[22.5, 72.0],
                           [31.4, 45.0],
                           [19.8, 85.0],
                           [27.6, 63]])

train_labels = np.array([[95],
                        [210],
                        [70],
                        [155]])

### 验证数据：特征、标签

In [3]:
test_features = np.array([[28.1, 58.0]])
test_labels = np.array([[165]])

## 模型

### 参数：权重、偏差

In [4]:
weight = np.ones([1, 2]) / 2
bias = np.zeros(1)

### 推理函数（前向传播）

In [5]:
def forward(x, w, b):
    return x @ w.T + b

### 损失函数（平均平方差）

In [6]:
def mse_loss(p, y):
    return ((p - y) ** 2).mean()

### 梯度函数

为了适应批次，梯度函数返回梯度的平均值，作为整个批次的梯度。

In [7]:
def gradient(p, y):
    return 2 * (p - y) / len(y)

### 超参数：学习率

In [8]:
LEARNING_RATE = 0.00001

### 反向函数（反向传播）

In [9]:
def backward(x, d, w, b):
    w -= d.T @ x * LEARNING_RATE
    b -= np.sum(d, axis=0) * LEARNING_RATE
    return w, b

## 训练

### 超参数：批大小

作为我们的第二个超参数，**批大小**（Batch Size）定义了每个批次所采用的训练数据的数量。

In [10]:
BATCH_SIZE = 2

### 迭代

从下面的代码可以看出，采用了小批次训练后，可以更快地完成一轮迭代。

In [11]:
for i in range(0, len(train_features), BATCH_SIZE):
    # 每次读入一个批次需要的多个训练数据：特征、标签
    features = train_features[i: i + BATCH_SIZE]
    labels = train_labels[i: i + BATCH_SIZE]

    # 同时对多个训练数据进行推理
    predictions = forward(features, weight, bias)
    # 计算多个训练数据的梯度平均值
    delta = gradient(predictions, labels)
    # 用梯度平均值更新一次参数
    weight, bias = backward(features, delta, weight, bias)

print(f"weight: {weight}")
print(f"bias: {bias}")

weight: [[0.59388172 0.68104165]]
bias: [0.00327249]


## 验证

### 推理

In [12]:
predictions = forward(test_features, weight, bias)

print(f'predictions: {predictions}')

predictions: [[56.19176426]]


### 评估

In [13]:
error = mse_loss(predictions, test_labels)

print(f'error: {error}')

error: 11839.232164432306


从验证的结果看，小批次训练一轮迭代的次数更少，但是收敛速度也相应地稍差。